In [2]:
!pip install pyserini
!pip install faiss-cpu
!git clone https://github.com/castorini/anserini.git --recurse-submodules

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 130.5 MB 27 kB/s 
     |████████████████████████████████| 2.0 MB 45.3 MB/s 
     |████████████████████████████████| 1.3 MB 46.6 MB/s 
     |████████████████████████████████| 1.5 MB 53.9 MB/s 
     |████████████████████████████████| 13.4 MB 38.6 MB/s 
     |████████████████████████████████| 4.5 MB 41.5 MB/s 
     |████████████████████████████████| 5.8 MB 40.0 MB/s 
     |████████████████████████████████| 188 kB 58.1 MB/s 
     |████████████████████████████████| 46 kB 4.4 MB/s 
     |████████████████████████████████| 7.6 MB 42.0 MB/s 
     |████████████████████████████████| 182 kB 63.1 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3
Looking in indexes: https://pypi.org/simple, htt

In [3]:
!wget https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2020-07-16.tar.gz  -P data/cord19_corpus/

--2022-12-08 08:11:39--  https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/cord-19_2020-07-16.tar.gz
Resolving ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)... 52.92.192.10, 52.92.164.138, 52.92.249.186, ...
Connecting to ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com (ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com)|52.92.192.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3662861028 (3.4G) [binary/octet-stream]
Saving to: ‘data/cord19_corpus/cord-19_2020-07-16.tar.gz’

cord-19_2020-07-16. 100%[===================>]   3.41G  32.3MB/s    in 1m 51s  

2022-12-08 08:13:30 (31.6 MB/s) - ‘data/cord19_corpus/cord-19_2020-07-16.tar.gz’ saved [3662861028/3662861028]



In [4]:
!ls data/cord19_corpus/ 
!tar -xvf "/content/data/cord19_corpus/cord-19_2020-07-16.tar.gz" -C "data/cord19_corpus"

cord-19_2020-07-16.tar.gz
2020-07-16/metadata.csv
2020-07-16/document_parses.tar.gz
2020-07-16/changelog
2020-07-16/cord_19_embeddings.tar.gz


In [5]:
!tar -xvf "/content/data/cord19_corpus/2020-07-16/document_parses.tar.gz" -C "data/cord19_corpus"

Streaming output truncated to the last 5000 lines.
document_parses/pdf_json/75cc9a3d0d2fc4baea735c4090944046019c8d39.json
document_parses/pmc_json/PMC2778948.xml.json
document_parses/pdf_json/44ffefb264faf5a03e45887a5694fbd07831f303.json
document_parses/pmc_json/PMC7298881.xml.json
document_parses/pdf_json/b5a7acc938d3d05c5ef67cb6b1dd090805e487e5.json
document_parses/pmc_json/PMC4177705.xml.json
document_parses/pdf_json/f620781b4f505d06e58fce2858339994c2f6b812.json
document_parses/pmc_json/PMC7103946.xml.json
document_parses/pdf_json/95528cdd92fb7dcccd02a8fa1294ca69524e1975.json
document_parses/pmc_json/PMC7286232.xml.json
document_parses/pdf_json/b8a7ae4513bce1715847acb4242d6be9396ab974.json
document_parses/pdf_json/230bc38351c778983c30364b4c5d1e725f00b785.json
document_parses/pmc_json/PMC7262020.xml.json
document_parses/pmc_json/PMC7267231.xml.json
document_parses/pdf_json/7991d32f2ab758697ca6d87a726ee826bc0dd5e4.json
document_parses/pmc_json/PMC7088309.xml.json
document_parses/pdf_j

In [7]:
import json
import os

json_files = os.listdir("/content/data/cord19_corpus/document_parses/pdf_json")

cleaned_data = {}

for i in range(len(json_files)):

  f = open("data/cord19_corpus/document_parses/pdf_json/{}".format(json_files[i]), 'r')
  data = json.load(f)

  if data['abstract'] == []:
    continue

  else:
    cleaned_data['id'] = data['paper_id']
    cleaned_data['contents'] = data['abstract'][0]['text']

    with open("data/cord19_corpus/new_document_parses/{}".format(json_files[i]), "w") as new_json:
      json.dump(cleaned_data, new_json)

In [8]:
!python -m pyserini.index.lucene -collection JsonCollection -generator DefaultLuceneDocumentGenerator -threads 9 \
-input /content/data/cord19_corpus/new_document_parses -index indexes/lucene-index-msmarco-passage -storePositions -storeDocvectors -storeRaw

Streaming output truncated to the last 5000 lines.
2022-12-08 08:26:20,470 DEBUG [pool-2-thread-7] index.IndexCollection$LocalIndexerThread (IndexCollection.java:215) - new_document_parses/598ce9971147cad544a81272a7b1eaee18630aee.json: 1 docs added.
2022-12-08 08:26:20,470 DEBUG [pool-2-thread-7] index.IndexCollection$LocalIndexerThread (IndexCollection.java:215) - new_document_parses/5113521b5a193540f5783167133b6eef32259c98.json: 1 docs added.
2022-12-08 08:26:20,471 DEBUG [pool-2-thread-7] index.IndexCollection$LocalIndexerThread (IndexCollection.java:215) - new_document_parses/f88cba884a640e2b82fafafdbb1deb992d8f22d3.json: 1 docs added.
2022-12-08 08:26:20,471 DEBUG [pool-2-thread-7] index.IndexCollection$LocalIndexerThread (IndexCollection.java:215) - new_document_parses/fa0c32f39abb602e0161bf738b5292e251c0c885.json: 1 docs added.
2022-12-08 08:26:20,471 DEBUG [pool-2-thread-7] index.IndexCollection$LocalIndexerThread (IndexCollection.java:215) - new_document_parses/491d6cddfee8cd2

In [42]:
import xml.etree.ElementTree as ET 

# Pass the path of the xml document 
tree = ET.parse('/content/topics-rnd5.xml')

queries = {}

for i in range(len(tree.getroot())):
  queries[tree.getroot()[i].attrib['number']] = tree.getroot()[i][1].text

queries

{'1': 'what is the origin of COVID-19',
 '2': 'how does the coronavirus respond to changes in the weather',
 '3': 'will SARS-CoV2 infected people develop immunity? Is cross protection possible?',
 '4': 'what causes death from Covid-19?',
 '5': 'what drugs have been active against SARS-CoV or SARS-CoV-2 in animal studies?',
 '6': 'what types of rapid testing for Covid-19 have been developed?',
 '7': 'are there serological tests that detect antibodies to coronavirus?',
 '8': 'how has lack of testing availability led to underreporting of true incidence of Covid-19?',
 '9': 'how has COVID-19 affected Canada',
 '10': 'has social distancing had an impact on slowing the spread of COVID-19?',
 '11': 'what are the guidelines for triaging patients infected with coronavirus?',
 '12': 'what are best practices in hospitals and at home in maintaining quarantine?',
 '13': 'what are the transmission routes of coronavirus?',
 '14': 'what evidence is there related to COVID-19 super spreaders',
 '15': 'h

In [43]:
from pyserini.index import IndexReader

index_reader = IndexReader('indexes/lucene-index-msmarco-passage')

In [44]:
index_reader.stats()

{'total_terms': 6386697,
 'documents': 58164,
 'non_empty_documents': 58159,
 'unique_terms': -1}

In [45]:
len_C = index_reader.stats()['total_terms']

In [46]:
def len_doc(d):
    doc_vector = index_reader.get_document_vector(d)

    len_d = 0
    for value in doc_vector.values():
      len_d += value
    
    return len_d

In [54]:
def coll_freq(t):
    try:
      cf = index_reader.get_term_counts(t)[1]
    except:
      cf = 0
    return cf

In [55]:
def term_freq(t, d):
    doc = index_reader.get_document_vector(d)
    
    if t in doc: tf = doc[t]
    else: tf = 0
    
    return tf

In [56]:
def prob_t_Md(t, d, lambd):

    p_t_Md = (1-lambd)*(term_freq(t, d) / abs(len_doc(d))) + lambd*(coll_freq(t) / abs(len_C))

    return p_t_Md


In [57]:
import math

def score_doc(q, d, lambd):

    p_q_Md_total = 0

    for term in q:

      p_q_Md_single = prob_t_Md(term, d, lambd)

      if p_q_Md_single == 0:
        continue

      else:
        p_q_Md_total += math.log(p_q_Md_single) 

    return p_q_Md_total
         

In [61]:
from pyserini.search.lucene import LuceneSearcher

results = []
searcher = LuceneSearcher("indexes/lucene-index-msmarco-passage")

lambd = 0.1

for qid, q in queries.items():

    hits = searcher.search(q, k=1000)
    query = index_reader.analyze(q)

    for hit in hits:
      final_score = score_doc(query, hit.docid, lambd)
      results.append([qid, hit.docid, final_score])   

In [62]:
check = set()
for res in results:
    if ((res[0], res[1])) in check:
        raise Exception("Error: Duplicate query-doc is found", res[0], res[1])
    check.add((res[0], res[1]))

# write results in a file
output_str = "\n".join([l[0] + "\tQ0\t" + l[1] + "\t0\t" + str(l[2]) + "\tlm_jm" for l in results])
open("lm_jm.run", "w").write(output_str)

3699465